In [21]:
import sys
from rmc_tools import rmc6f_stuff
import copy

In [10]:
rmc6f_file_name = "ceramic.rmc6f"
lili_ul = 5.0
lita_ul = 3.0

input_rmc_config = rmc6f_stuff.RMC6FReader(rmc6f_file_name)


Reading in the RMC6F configuration...
Progress: 
....5%....10%....15%....20%
....25%....30%....35%....40%
....45%....50%....55%....60%
....65%....70%....75%....80%
....85%....90%....95%....100%

------------------------------------------
RMC6F configuration successfully read in.
Time taken:      0.217 s
------------------------------------------


In [15]:
all_neigh = dict()
key_combo_at = dict()

for i in range(input_rmc_config.numAtoms):
    line_s = input_rmc_config.atomsLine[i].strip()
    atom_key_combo = line_s.split()[-3] + "-"
    atom_key_combo += (line_s.split()[-2] + "-")
    atom_key_combo += (line_s.split()[-1] + "-")
    atom_key_combo += (line_s.split()[-4])
    key_combo_at[atom_key_combo] = input_rmc_config.atomsEle[i]
    
    if input_rmc_config.atomsEle[i] == "Li":
        atom_coord_1 = input_rmc_config.atomsCoordInt[i]
        all_neigh[atom_key_combo] = list()
        for j in range(input_rmc_config.numAtoms):
            cond1 = input_rmc_config.atomsEle[j] == "Li"
            cond2 = input_rmc_config.atomsEle[j] == "Ta"
            if cond1 or cond2:
                line_s = input_rmc_config.atomsLine[j].strip()
                atom_key_combo1 = line_s.split()[-3] + "-"
                atom_key_combo1 += (line_s.split()[-2] + "-")
                atom_key_combo1 += (line_s.split()[-1] + "-")
                atom_key_combo1 += (line_s.split()[-4])

                atom_coord_2 = input_rmc_config.atomsCoordInt[j]
                dist_temp = rmc6f_stuff.dist_calc_coord(
                    atom_coord_1, atom_coord_2,
                    input_rmc_config.vectors
                )
                if cond1 and dist_temp <= lili_ul and dist_temp > 1.E-5:
                    all_neigh[atom_key_combo].append(atom_key_combo1)
                if cond2 and dist_temp <= lita_ul and dist_temp > 1.E-5:
                    all_neigh[atom_key_combo].append(atom_key_combo1)

In [29]:
all_neigh_final = copy.deepcopy(all_neigh)
for key, item in all_neigh.items():
    for key_key in item:
        if key_combo_at[key_key] == "Li":
            neigh_tmp = all_neigh[key_key]
            common_list = list(set(item) & set(neigh_tmp))
            num_cm_ta = 0
            for common_site in common_list:
                if key_combo_at[common_site] == "Ta":
                    num_cm_ta += 1
            if num_cm_ta > 0:
                all_neigh_final[key].remove(key_key)

In [30]:
for key, item in all_neigh_final.items():
    for key_key in item:
        if key_combo_at[key_key] == "Ta":
            all_neigh_final[key].remove(key_key)